In [ ]:
import cv2 as cv  # Import OpenCV library for computer vision tasks
import matplotlib.pyplot as plt  # Import Matplotlib for plotting (not used in this code)
import time  # Import time library for timing operations

# Load the pre-trained pose estimation model from a TensorFlow file
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

# Set input dimensions for the model
inWidth = 368
inHeight = 368
thr = 0.2  # Confidence threshold for detecting body parts

# Define body parts and their corresponding indices
BODY_PARTS = {
    "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
    "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
    "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
    "LEye": 15
}

# Define pairs of body parts to connect with lines
POSE_PAIRS = [
    ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
    ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
    ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"],
    ["Neck", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"],
    ["Neck", "Nose"], ["Nose", "REye"], ["Nose", "LEye"]
]

# Function to perform pose estimation on a given frame
def pos_estimation(frame):
    frameWidth = frame.shape[1]  # Get the width of the frame
    frameHeight = frame.shape[0]  # Get the height of the frame

    # Prepare the frame for the model
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))

    # Forward pass through the network
    out = net.forward()
    out = out[:, :19, :, :]  # Keep only the first 19 output channels

    assert (len(BODY_PARTS) <= out.shape[1])  # Ensure the output matches the body parts

    points = []  # List to store detected points

    # Loop through each body part to find its location
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]  # Get the heatmap for the current body part

        # Find the maximum confidence and its location
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]  # Scale x coordinate
        y = (frameHeight * point[1]) / out.shape[2]  # Scale y coordinate

        # Append the point if confidence is above the threshold
        points.append((int(x), int(y)) if conf > thr else None)

    # Draw lines and points for detected body parts
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert (partFrom in BODY_PARTS)
        assert (partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        # Draw the line and points if both body parts are detected
        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)  # Draw line
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)  # Draw point
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)  # Draw point

    # Measure the time taken for the forward pass
    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))  # Display time

    return frame  # Return the processed frame

# Capture video from the webcam
cap = cv.VideoCapture(0)

start_time = time.time()  # Start timing
elapsed_time = 0  # Initialize elapsed time
while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    if not ret:  # Break if no frame is captured
        break

    # Perform pose estimation on the current frame
    estimated_frame = pos_estimation(frame)

    # Display the processed frame
    cv.imshow('Pose Estimation', estimated_frame)

    # Check for 'p' key press to quit the application
    if cv.waitKey(1) & 0xFF == ord('p'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv.destroyAllWindows()
